In [13]:
# Install Python packages (keep this, it installs selenium and webdriver_manager)
!pip install yfinance requests bs4 plotly selenium webdriver_manager

# Install Google Chrome stable (instead of apt-get chromium-browser/chromedriver)
# This provides a more recent and stable Chrome version for Selenium
!wget https://dl.google.com/linux/direct/google-chrome-stable_current_amd64.deb
!dpkg -i google-chrome-stable_current_amd64.deb
!apt-get install -f # Fix broken dependencies that might arise from dpkg

Hit:1 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:2 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [128 kB]
Get:3 http://security.ubuntu.com/ubuntu jammy-security InRelease [129 kB]
Hit:4 https://cli.github.com/packages stable InRelease
Get:5 http://archive.ubuntu.com/ubuntu jammy-backports InRelease [127 kB]
Get:6 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,632 B]
Get:7 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease [1,581 B]
Get:8 https://r2u.stat.illinois.edu/ubuntu jammy InRelease [6,555 B]
Get:9 http://security.ubuntu.com/ubuntu jammy-security/restricted amd64 Packages [6,008 kB]
Get:10 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease [18.1 kB]
Hit:11 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Get:12 http://security.ubuntu.com/ubuntu jammy-security/main amd64 Packages [3,535 kB]
Get:13 http://security.ubuntu.com/ubuntu jammy-security/univers

In [11]:
# ======= SETUP (Run once at top of notebook) =======
# If in Colab uncomment the install line
# !pip install yfinance requests beautifulsoup4 plotly selenium webdriver_manager

import yfinance as yf
import pandas as pd
import requests
from bs4 import BeautifulSoup
import plotly.graph_objects as go
from datetime import datetime

# For Selenium
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.chrome.options import Options

pd.set_option('display.max_rows', 20)
pd.set_option('display.max_columns', 50)

In [3]:
# Q1: Tesla stock data using yfinance
# Screenshot: take 1) code screenshot and 2) screenshot of the printed DataFrame below (first 5-10 rows)

tesla_ticker = "TSLA"
tesla_df = yf.download(tesla_ticker, start="2010-01-01", end=datetime.today().strftime('%Y-%m-%d'))
tesla_df.reset_index(inplace=True)
# show first rows for screenshot
tesla_df.head(10)


/tmp/ipython-input-1890215419.py:5: FutureWarning: YF.download() has changed argument auto_adjust default to True
  tesla_df = yf.download(tesla_ticker, start="2010-01-01", end=datetime.today().strftime('%Y-%m-%d'))
[*********************100%***********************]  1 of 1 completed


Price,Date,Close,High,Low,Open,Volume
Ticker,,TSLA,TSLA,TSLA,TSLA,TSLA
0,2010-06-29,1.592667,1.666667,1.169333,1.266667,281494500
1,2010-06-30,1.588667,2.028000,1.553333,1.719333,257806500
2,2010-07-01,1.464000,1.728000,1.351333,1.666667,123282000
3,2010-07-02,1.280000,1.540000,1.247333,1.533333,77097000
4,2010-07-06,1.074000,1.333333,1.055333,1.333333,103003500
5,2010-07-07,1.053333,1.108667,0.998667,1.093333,103825500
6,2010-07-08,1.164000,1.168000,1.038000,1.076000,115671000
7,2010-07-09,1.160000,1.193333,1.103333,1.172000,60759000
8,2010-07-12,1.136667,1.204667,1.133333,1.196667,33037500


In [22]:
# Run once if needed in Colab:
# !pip install yfinance requests beautifulsoup4

import yfinance as yf
import pandas as pd
import requests
from bs4 import BeautifulSoup
from datetime import datetime
import traceback

def fetch_ticker_quarterly_revenue(ticker_symbol, verbose=True):
    """
    Returns a DataFrame with columns ['Date','Revenue'] (Revenue in USD as float)
    Strategy:
      1) Try yfinance.Ticker(...).quarterly_financials -> look for 'Total Revenue' or 'Revenue'
      2) If (1) fails or is empty, try scraping macrotrends.net robustly
    """
    # Helper to tidy DataFrame
    def tidy_df_from_series(series):
        # series: pandas Series where index are Periods/datetimes and values are numbers
        try:
            df = series.reset_index()
            df.columns = ['Date', 'Revenue']
            # convert Period/Index to datetime if possible
            df['Date'] = pd.to_datetime(df['Date'].astype(str), errors='coerce')
            # Ensure revenue numeric
            df['Revenue'] = pd.to_numeric(df['Revenue'], errors='coerce')
            df = df.dropna(subset=['Revenue'])
            # sort by Date ascending
            df = df.sort_values('Date').reset_index(drop=True)
            return df
        except Exception:
            return None

    # 1) Try yfinance
    try:
        if verbose: print(f"Trying yfinance for {ticker_symbol} ...")
        tk = yf.Ticker(ticker_symbol)
        # Preferred keys: 'Total Revenue', 'Revenue', 'totalRevenue'
        qfin = tk.quarterly_financials  # columns are periods, rows are line items
        if isinstance(qfin, pd.DataFrame) and not qfin.empty:
            # try multiple possible row names
            candidates = [c for c in qfin.index.astype(str)]
            # lower-case candidate matching
            row_match = None
            for pattern in ['total revenue', 'totalrevenues', 'revenue', 'totalRevenue', 'Total Revenue', 'TotalRevenue']:
                for r in candidates:
                    if pattern.lower() in r.lower().replace(' ', '') or pattern.lower().replace(' ', '') in r.lower().replace(' ', ''):
                        row_match = r
                        break
                if row_match:
                    break
            if not row_match:
                # Also check the 'Net Revenue' or similar
                for r in candidates:
                    if 'revenue' in r.lower():
                        row_match = r
                        break

            if row_match:
                if verbose: print("yfinance provided quarterly_financials; extracting revenue row:", row_match)
                revenue_series = qfin.loc[row_match]
                revenue_df = tidy_df_from_series(revenue_series)
                if revenue_df is not None and not revenue_df.empty:
                    # yfinance values are floats (USD). Return.
                    return revenue_df
            else:
                if verbose: print("yfinance quarterly_financials present but no revenue-like row found.")
        else:
            if verbose: print("yfinance quarterly_financials empty or unavailable.")
    except Exception as e:
        if verbose:
            print("yfinance attempt raised an exception:")
            traceback.print_exc()

    # 2) Fallback: macrotrends scraping (robust)
    try:
        if verbose: print("Falling back to macrotrends scraping ...")
        url = f"https://www.macrotrends.net/stocks/charts/{ticker_symbol}/{ticker_symbol.lower()}/revenue"
        headers = {
            "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/115.0 Safari/537.36",
            "Accept-Language": "en-US,en;q=0.9"
        }
        resp = requests.get(url, headers=headers, timeout=15)
        html = resp.text

        soup = BeautifulSoup(html, "html.parser")
        tables = soup.find_all("table")
        if verbose: print(f"Found {len(tables)} <table> elements on macrotrends page")

        found = None
        for i, table in enumerate(tables):
            txt = table.get_text(separator=' ').strip()
            # we look for Revenue header and numbers with $ or commas
            if 'Revenue' in txt or '$' in txt or 'Quarterly Revenue' in txt:
                # attempt to read this table
                try:
                    df = pd.read_html(str(table))[0]
                except Exception:
                    continue
                # normalize column count
                if df.shape[1] >= 2:
                    # try to identify date and revenue columns (first two columns often)
                    df.columns = [str(c).strip() for c in df.columns]
                    # heuristics: pick first col as Date, pick col with 'Revenue' or $ as Rev
                    date_col = df.columns[0]
                    rev_col = None
                    for c in df.columns[1:]:
                        sample = df[c].astype(str).str.lower().head(10).to_list()
                        if any('revenue' in s for s in sample) or any('$' in s for s in sample) or any(',' in s for s in sample):
                            rev_col = c
                            break
                    if rev_col is None:
                        rev_col = df.columns[1]  # fallback
                    sub = df[[date_col, rev_col]].copy()
                    sub.columns = ['Date', 'Revenue']
                    # clean Revenue strings
                    sub['Revenue'] = sub['Revenue'].astype(str).str.replace(r'[\$,()]', '', regex=True).str.replace(r'\s*USD.*', '', regex=True).str.strip()
                    # drop empty or dash rows
                    sub = sub[sub['Revenue'].str.strip().replace('','-') != '-']
                    # keep only numeric-like revenue rows
                    sub['Revenue_clean'] = sub['Revenue'].str.replace(',', '').str.replace('—','').str.replace(' ', '')
                    # remove rows that are clearly non-numeric
                    sub = sub[sub['Revenue_clean'].str.replace('.', '', 1).str.isnumeric()]
                    if sub.empty:
                        continue
                    sub['Revenue'] = sub['Revenue_clean'].str.replace(',', '').astype(float)
                    sub = sub.drop(columns=['Revenue_clean'])
                    # try parse date
                    try:
                        sub['Date'] = pd.to_datetime(sub['Date'].astype(str), errors='coerce')
                    except:
                        pass
                    sub = sub.sort_values('Date').reset_index(drop=True)
                    found = sub
                    if verbose: print(f"macrotrends: using table index {i} as revenue table")
                    break

        if found is not None and not found.empty:
            return found
        else:
            if verbose: print("macrotrends fallback did not discover a usable revenue table.")
    except Exception:
        if verbose:
            print("macrotrends attempt raised an exception:")
            traceback.print_exc()

    # 3) If both fail
    raise RuntimeError(f"Could not fetch quarterly revenue for {ticker_symbol} via yfinance or macrotrends. "
                       "Possible causes: site blocking, network issues, or the ticker is delisted. "
                       "See verbose prints above for details.")

# Example usage:
try:
    tesla_rev_df = fetch_ticker_quarterly_revenue("TSLA", verbose=True)
    print("\n=== Result (first rows) ===")
    display(tesla_rev_df.head(10))
except Exception as e:
    print("ERROR:", e)


Trying yfinance for TSLA ...
yfinance provided quarterly_financials; extracting revenue row: Total Revenue

=== Result (first rows) ===


,Date,Revenue
0,2024-09-30,2.518200e+10
1,2024-12-31,2.570700e+10
2,2025-03-31,1.933500e+10
3,2025-06-30,2.249600e+10
4,2025-09-30,2.809500e+10


In [23]:
# Q3: GameStop stock data using yfinance
# Screenshot: code + output (DataFrame head)

gme_ticker = "GME"
gme_df = yf.download(gme_ticker, start="2010-01-01", end=datetime.today().strftime('%Y-%m-%d'))
gme_df.reset_index(inplace=True)
gme_df.head(10)


/tmp/ipython-input-70292266.py:5: FutureWarning: YF.download() has changed argument auto_adjust default to True
  gme_df = yf.download(gme_ticker, start="2010-01-01", end=datetime.today().strftime('%Y-%m-%d'))
[*********************100%***********************]  1 of 1 completed


Price,Date,Close,High,Low,Open,Volume
Ticker,,GME,GME,GME,GME,GME
0,2010-01-04,3.854644,3.863060,3.703151,3.714934,26702800
1,2010-01-05,3.959005,3.996036,3.854643,3.856327,21269600
2,2010-01-06,4.044850,4.056633,3.908506,3.948904,21471200
3,2010-01-07,3.443929,3.563440,3.268872,3.368183,164761200
4,2010-01-08,3.415315,3.573541,3.405216,3.474229,47872400
5,2010-01-11,3.420365,3.465813,3.385017,3.457397,28085200
6,2010-01-12,3.496110,3.501160,3.416997,3.477594,28486000
7,2010-01-13,3.499478,3.517993,3.470863,3.506211,17787600
8,2010-01-14,3.450664,3.506211,3.432148,3.487695,27087600


In [25]:
# Q4: Robust GameStop (GME) quarterly revenue fetch — copy & run whole cell
# Screenshot: after running, take a screenshot showing this code cell and the printed DataFrame (head)

# If you haven't already run setup, uncomment:
# !pip install yfinance requests beautifulsoup4

import yfinance as yf
import pandas as pd
import requests
from bs4 import BeautifulSoup
import traceback

def fetch_ticker_quarterly_revenue(ticker_symbol, verbose=True):
    def tidy_df_from_series(series):
        try:
            df = series.reset_index()
            df.columns = ['Date', 'Revenue']
            df['Date'] = pd.to_datetime(df['Date'].astype(str), errors='coerce')
            df['Revenue'] = pd.to_numeric(df['Revenue'], errors='coerce')
            df = df.dropna(subset=['Revenue']).sort_values('Date').reset_index(drop=True)
            return df
        except Exception:
            return None

    # 1) Try yfinance
    try:
        if verbose: print(f"Trying yfinance for {ticker_symbol} ...")
        tk = yf.Ticker(ticker_symbol)
        qfin = tk.quarterly_financials
        if isinstance(qfin, pd.DataFrame) and not qfin.empty:
            candidates = [str(c) for c in qfin.index]
            row_match = None
            for pattern in ['total revenue', 'totalrevenues', 'revenue', 'totalRevenue', 'Total Revenue', 'TotalRevenue']:
                for r in candidates:
                    if pattern.lower().replace(' ', '') in r.lower().replace(' ', ''):
                        row_match = r
                        break
                if row_match: break
            if not row_match:
                for r in candidates:
                    if 'revenue' in r.lower():
                        row_match = r
                        break
            if row_match:
                if verbose: print("yfinance: extracting row:", row_match)
                revenue_series = qfin.loc[row_match]
                revenue_df = tidy_df_from_series(revenue_series)
                if revenue_df is not None and not revenue_df.empty:
                    return revenue_df
            else:
                if verbose: print("yfinance quarterly_financials present but no revenue-like row found.")
        else:
            if verbose: print("yfinance quarterly_financials empty or unavailable.")
    except Exception:
        if verbose:
            print("yfinance attempt raised an exception:")
            traceback.print_exc()

    # 2) Fallback: macrotrends scraping
    try:
        if verbose: print("Falling back to macrotrends scraping ...")
        url = f"https://www.macrotrends.net/stocks/charts/{ticker_symbol}/{ticker_symbol.lower()}/revenue"
        headers = {
            "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/115.0 Safari/537.36",
            "Accept-Language": "en-US,en;q=0.9"
        }
        resp = requests.get(url, headers=headers, timeout=15)
        html = resp.text
        soup = BeautifulSoup(html, "html.parser")
        tables = soup.find_all("table")
        if verbose: print(f"Found {len(tables)} <table> elements on macrotrends page")

        found = None
        for i, table in enumerate(tables):
            txt = table.get_text(separator=' ').strip()
            if 'Revenue' in txt or '$' in txt or 'Quarterly Revenue' in txt:
                try:
                    df = pd.read_html(str(table))[0]
                except Exception:
                    continue
                if df.shape[1] >= 2:
                    df.columns = [str(c).strip() for c in df.columns]
                    date_col = df.columns[0]
                    rev_col = None
                    for c in df.columns[1:]:
                        sample = df[c].astype(str).str.lower().head(10).to_list()
                        if any('revenue' in s for s in sample) or any('$' in s for s in sample) or any(',' in s for s in sample):
                            rev_col = c
                            break
                    if rev_col is None:
                        rev_col = df.columns[1]
                    sub = df[[date_col, rev_col]].copy()
                    sub.columns = ['Date', 'Revenue']
                    sub['Revenue'] = sub['Revenue'].astype(str).str.replace(r'[\$,()]', '', regex=True).str.replace(r'\s*USD.*', '', regex=True).str.strip()
                    sub = sub[sub['Revenue'].str.strip().replace('','-') != '-']
                    sub['Revenue_clean'] = sub['Revenue'].str.replace(',', '').str.replace('—','').str.replace(' ', '')
                    sub = sub[sub['Revenue_clean'].str.replace('.', '', 1).str.isnumeric()]
                    if sub.empty:
                        continue
                    sub['Revenue'] = sub['Revenue_clean'].str.replace(',', '').astype(float)
                    sub = sub.drop(columns=['Revenue_clean'])
                    try:
                        sub['Date'] = pd.to_datetime(sub['Date'].astype(str), errors='coerce')
                    except:
                        pass
                    sub = sub.sort_values('Date').reset_index(drop=True)
                    found = sub
                    if verbose: print(f"macrotrends: using table index {i} as revenue table")
                    break

        if found is not None and not found.empty:
            return found
        else:
            if verbose: print("macrotrends fallback did not discover a usable revenue table.")
    except Exception:
        if verbose:
            print("macrotrends attempt raised an exception:")
            traceback.print_exc()

    raise RuntimeError(f"Could not fetch quarterly revenue for {ticker_symbol} via yfinance or macrotrends.")

# ==== Run for GME ====
try:
    gme_rev_df = fetch_ticker_quarterly_revenue("GME", verbose=True)
    print("\n=== GME Revenue (first 10 rows) ===")
    display(gme_rev_df.head(10))
except Exception as e:
    print("ERROR:", e)


Trying yfinance for GME ...
yfinance: extracting row: Total Revenue

=== GME Revenue (first 10 rows) ===


,Date,Revenue
0,2024-07-31,7.983000e+08
1,2024-10-31,8.603000e+08
2,2025-01-31,1.282600e+09
3,2025-04-30,7.324000e+08
4,2025-07-31,9.722000e+08


In [26]:
# Q5: Tesla Dashboard (Stock price and Revenue)
# Screenshot: take 2 screenshots: 1) stock price chart visible, 2) revenue chart visible
# Note: in notebooks, fig.show() will display. Take screenshots of displayed figures.

# Tesla Stock Price (Close)
fig_tesla_price = go.Figure()
fig_tesla_price.add_trace(go.Scatter(x=tesla_df['Date'], y=tesla_df['Close'], name='Tesla Close Price', mode='lines'))
fig_tesla_price.update_layout(title='Tesla Close Price over Time', xaxis_title='Date', yaxis_title='Price (USD)')
fig_tesla_price.show()

# Tesla Quarterly Revenue
# If revenue Date not parsed as datetime, use as-is
x_rev = tesla_rev['Date']
try:
    x_rev = pd.to_datetime(x_rev)
except:
    pass

fig_tesla_rev = go.Figure()
fig_tesla_rev.add_trace(go.Bar(x=x_rev, y=tesla_rev['Revenue'], name='Tesla Revenue'))
fig_tesla_rev.update_layout(title='Tesla Quarterly Revenue', xaxis_title='Date', yaxis_title='Revenue (USD)')
fig_tesla_rev.show()


TypeError: 'NoneType' object is not subscriptable

In [27]:
# Q6: GameStop Dashboard (Stock price and Revenue)
# Screenshot: take 2 screenshots: 1) stock price chart, 2) revenue chart

# GameStop Stock Price (Close)
fig_gme_price = go.Figure()
fig_gme_price.add_trace(go.Scatter(x=gme_df['Date'], y=gme_df['Close'], name='GME Close Price', mode='lines'))
fig_gme_price.update_layout(title='GameStop Close Price over Time', xaxis_title='Date', yaxis_title='Price (USD)')
fig_gme_price.show()

# GameStop Quarterly Revenue
x_rev_gme = gme_rev['Date']
try:
    x_rev_gme = pd.to_datetime(x_rev_gme)
except:
    pass

fig_gme_rev = go.Figure()
fig_gme_rev.add_trace(go.Bar(x=x_rev_gme, y=gme_rev['Revenue'], name='GME Revenue'))
fig_gme_rev.update_layout(title='GameStop Quarterly Revenue', xaxis_title='Date', yaxis_title='Revenue (USD)')
fig_gme_rev.show()


NameError: name 'gme_rev' is not defined

In [29]:
# Q7: Notebook saving & submission instructions
# Screenshot: You must take a screenshot AFTER uploading your .ipynb file to Coursera.

from google.colab import files

print("✔ Your project files will now be prepared for download...")

# Safe saving: use try/except so your notebook doesn't crash if any DF is missing

try:
    tesla_df.to_csv("tesla_stock.csv", index=False)
    print("Saved: tesla_stock.csv")
except:
    print("Could not save tesla_stock.csv (data missing)")

try:
    tesla_rev.to_csv("tesla_revenue.csv", index=False)
    print("Saved: tesla_revenue.csv")
except:
    print("Could not save tesla_revenue.csv (data missing)")

try:
    gme_df.to_csv("gme_stock.csv", index=False)
    print("Saved: gme_stock.csv")
except:
    print("Could not save gme_stock.csv (data missing)")

try:
    gme_rev.to_csv("gme_revenue.csv", index=False)
    print("Saved: gme_revenue.csv")
except:
    print("Could not save gme_revenue.csv (data missing)")

print("\n✔ Now do the final step:")
print("Go to: File → Download → Download .ipynb")
print("Then upload your notebook file to Coursera.")


✔ Your project files will now be prepared for download...
Saved: tesla_stock.csv
Could not save tesla_revenue.csv (data missing)
Saved: gme_stock.csv
Could not save gme_revenue.csv (data missing)

✔ Now do the final step:
Go to: File → Download → Download .ipynb
Then upload your notebook file to Coursera.
